In [3]:
import os

In [4]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises'

In [7]:
import pandas as pd

In [9]:
data = pd.read_csv("artifacts/data_ingestion/allmetrics.csv")
data.head()

,CPU_Load,Memory_Usage,Disk_Usage,Network_IO,Server_Load,Power_Consumption,Temperature,Num_of_Running_Processes,Storage_Capacity,Server_Uptime,Status,Server_ID,Server_Dependency
0,52.629190,48.885554,65.686480,42.113656,48.569130,39.249038,55.792941,51,56.160407,48.473423,59.473216,Server_1,Server_2945
1,56.634583,20.232022,35.264304,37.050711,66.371983,50.963175,38.899316,46,50.962167,58.002286,39.901573,Server_2,Server_738
2,34.024515,21.227805,56.550348,34.393895,45.425457,53.193483,29.091559,48,59.323921,50.566179,41.766745,Server_3,Server_1396
3,43.595644,42.301180,47.927740,59.211128,58.502400,31.574986,49.276095,50,55.188524,49.681689,37.464329,Server_4,Server_242
4,45.944282,58.220190,58.248009,66.926232,45.186706,50.537771,45.028851,61,71.671269,71.834936,54.728673,Server_5,Server_2681


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CPU_Load                  5000 non-null   float64
 1   Memory_Usage              5000 non-null   float64
 2   Disk_Usage                5000 non-null   float64
 3   Network_IO                5000 non-null   float64
 4   Server_Load               5000 non-null   float64
 5   Power_Consumption         5000 non-null   float64
 6   Temperature               5000 non-null   float64
 7   Num_of_Running_Processes  5000 non-null   int64  
 8   Storage_Capacity          5000 non-null   float64
 9   Server_Uptime             5000 non-null   float64
 10  Status                    5000 non-null   float64
 11  Server_ID                 5000 non-null   object 
 12  Server_Dependency         5000 non-null   object 
dtypes: float64(10), int64(1), object(2)
memory usage: 507.9+ KB


In [11]:
data.isnull().sum()

CPU_Load                    0
Memory_Usage                0
Disk_Usage                  0
Network_IO                  0
Server_Load                 0
Power_Consumption           0
Temperature                 0
Num_of_Running_Processes    0
Storage_Capacity            0
Server_Uptime               0
Status                      0
Server_ID                   0
Server_Dependency           0
dtype: int64

In [12]:
data.shape

(5000, 13)

In [13]:
data.columns

Index(['CPU_Load', 'Memory_Usage', 'Disk_Usage', 'Network_IO', 'Server_Load',
       'Power_Consumption', 'Temperature', 'Num_of_Running_Processes',
       'Storage_Capacity', 'Server_Uptime', 'Status', 'Server_ID',
       'Server_Dependency'],
      dtype='object')

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [17]:
from data_it_entp.constants import *
from data_it_entp.utils.common import read_yaml,create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config


In [19]:
import os
from data_it_entp import logger

In [20]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [21]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-11-07 18:49:11,950:INFO:common:yaml file: config\config.yaml loaded successfully:]
[2023-11-07 18:49:11,950:INFO:common:yaml file: params.yaml loaded successfully:]
[2023-11-07 18:49:11,950:INFO:common:yaml file: schema.yaml loaded successfully:]
[2023-11-07 18:49:11,950:INFO:common:created directory at: artifacts:]
[2023-11-07 18:49:11,950:INFO:common:created directory at: artifacts/data_validation:]
